# DBSCAN Clustering
---

## 1. Introduction

Density-Based Spatial Clustering of Applications with Noise (DBSCAN) is an
unsupervised clustering algorithm that groups data points based on **local density**
rather than distance to a centroid.

Unlike K-Means, DBSCAN:

- Does not require specifying the number of clusters  
- Can discover arbitrarily shaped clusters  
- Explicitly identifies noise / outliers  
- Is robust to non-spherical cluster structure  

In this notebook, we apply DBSCAN using a **from-scratch implementation**
from the `rice_ml` package.

---

## 2. Mathematical Intuition Behind DBSCAN

DBSCAN relies on two hyperparameters:

- **$\varepsilon$ (epsilon)**: neighborhood radius  
- **`min_samples`**: minimum number of points required to form a dense region  

### $\varepsilon$-Neighborhood

For a point $x$, the $\varepsilon$-neighborhood is defined as:

$$
N_\varepsilon(x) = \{ y \mid \text{distance}(x, y) \le \varepsilon \}
$$

### Core Point

A point $x$ is a **core point** if:

$$
|N_\varepsilon(x)| \ge \text{min\_samples}
$$

### Border and Noise Points

- **Border point**: within $\varepsilon$ of a core point, but not dense itself  
- **Noise point**: not reachable from any core point  

Clusters are formed by **density reachability**: starting from a core point,
all reachable points are recursively added to the same cluster.

## Two Moons (Cluster Moons) Dataset

The Two Moons dataset is a synthetic benchmark dataset commonly used to
evaluate clustering algorithms on **non-linear and non-spherical structures**.

Each observation corresponds to a point in two-dimensional space. The data
forms two interleaving crescent-shaped clusters (“moons”), which violate the
assumptions of centroid-based clustering methods.

### Dataset Characteristics

- Designed to test non-linear clustering behavior
- Clusters are non-spherical and interleaved
- No centroid structure
- Well-suited for density-based methods such as DBSCAN

### Features

| Feature | Description |
|-------|-------------|
| x1 | First spatial coordinate |
| x2 | Second spatial coordinate |

All features are continuous and numeric.

### Structure

- Total samples: varies (commonly 200–500)
- Dimensionality: 2
- Cluster geometry: Non-linear, crescent-shaped
- Noise: Optional (depending on generation or source)

### Notes

- No missing values
- Ground-truth labels may be present but are **not used** for clustering
- Standardization is recommended prior to distance-based clustering

### Use Case for DBSCAN

The Two Moons dataset strongly favors DBSCAN because clusters are defined by
local density and connectivity rather than proximity to a centroid. This
dataset highlights DBSCAN’s ability to recover complex geometries that
centroid-based algorithms such as K-Means cannot represent.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from rice_ml.unsupervised_learning.dbscan import DBSCAN
from rice_ml.processing.preprocessing import standardize

df = pd.read_csv("cluster_moons.csv")
df.head()

FileNotFoundError: https://raw.githubusercontent.com/deric/clustering-benchmark/master/src/main/resources/datasets/artificial/Aggregation.txt not found.

## 4. Exploratory Data Analysis (EDA)

The goal of exploratory data analysis for DBSCAN is not to study feature
distributions or correlations, but to understand the **geometric and density
structure** of the data. Specifically, EDA should help answer:

- What does the spatial structure look like?
- Are clusters non-spherical?
- What density scales exist?
- How should the DBSCAN hyperparameters be chosen?

All distance-based analysis below uses the same Euclidean distance logic as
the clustering algorithm itself, ensuring consistency between EDA and model
behavior.

---

### Raw Data Visualization

We begin by visualizing the raw data in its original feature space.

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:, 1], s=20, alpha=0.7)
plt.title("Aggregation Dataset — Raw Data")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()



## Local Density Approximation (ε-ball Counts)

In [ ]:
def epsilon_density(X, eps):
    densities = np.zeros(len(X))
    for i in range(len(X)):
        distances = np.linalg.norm(X - X[i], axis=1)
        densities[i] = np.sum(distances <= eps)
    return densities

# Compute and visualize:

eps_candidate = 0.3
densities = epsilon_density(X_std, eps_candidate)

plt.figure(figsize=(6, 6))
plt.scatter(
    X_std[:, 0],
    X_std[:, 1],
    c=densities,
    cmap="viridis",
    s=25
)
plt.colorbar(label="Points within ε")
plt.title("Local Density Approximation")
plt.xlabel("Standardized x1")
plt.ylabel("Standardized x2")
plt.show()


### Interpretation

The scatter plot reveals **multiple dense regions** with irregular, non-spherical
shapes. There is no obvious centroid structure, and several points lie near
cluster boundaries. This immediately suggests that centroid-based methods such
as K-Means are inappropriate, while **density-based approaches like DBSCAN are
well-suited for this dataset**


### k-Distance Analysis (ε Selection)

A standard diagnostic for DBSCAN is the **k-distance plot**, where k is chosen
as min_samples - 1. This plot shows the distance from each point to its k-th
nearest neighbor.

We compute these distances using the custom KNN implementation from the
repository to remain consistent with the DBSCAN distance metric.

### Standardization Note

Although DBSCAN does not require scaling in theory, distance-based neighborhoods
are sensitive to feature scale. We therefore work with standardized features
in subsequent analysis.

In [ ]:
X_std = standardize(X)

from rice_ml.supervised_learning.knn import KNNClassifier

min_samples = 10
k = min_samples - 1

knn = KNNClassifier(n_neighbors=k, metric="euclidean")
knn.fit(X_std, np.zeros(len(X_std)))  # dummy labels

distances, _ = knn.kneighbors(X_std)
k_distances = np.sort(distances[:, -1])

plt.figure(figsize=(6, 4))
plt.plot(k_distances)
plt.title("k-Distance Plot (k = min_samples - 1)")
plt.xlabel("Sorted Points")
plt.ylabel("k-th Nearest Neighbor Distance")
plt.show()


### Interpretation

The k-distance curve exhibits a relatively flat region followed by a sharp
increase. The flat portion corresponds to points inside dense clusters, while
the steep rise indicates sparse regions and noise. The transition point (the
“elbow”) provides an empirical estimate for a suitable value of $\varepsilon$.
Choosing $\varepsilon$ near this elbow balances cluster completeness with noise
separation.

### Local Density Visualization via k-NN Distances

To better understand how density varies across the feature space, we color each
point by its distance to the k-th nearest neighbor. Smaller values indicate
denser regions.

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(
    X_std[:, 0],
    X_std[:, 1],
    c=distances[:, -1],
    cmap="viridis",
    s=25
)
plt.colorbar(label="k-th Nearest Neighbor Distance")
plt.title("Local Density Visualization")
plt.xlabel("Standardized x1")
plt.ylabel("Standardized x2")
plt.show()


## Interpretation

Dense cluster cores appear as darker regions (small k-NN distance), while lighter
regions correspond to boundaries and sparse areas. This visualization highlights
variation in density across clusters and explains why DBSCAN may label certain
boundary points as noise.

### Noise Sensitivity Preview

As a final EDA step, we apply DBSCAN using a relatively small $\varepsilon$ to
observe which points are most sensitive to density requirements.

In [ ]:
dbscan_preview = DBSCAN(eps=0.2, min_samples=10)
labels_preview = dbscan_preview.fit_predict(X_std)

plt.figure(figsize=(6, 5))
plt.scatter(
    X_std[:, 0],
    X_std[:, 1],
    c=labels_preview,
    cmap="tab10",
    s=20
)
plt.title("Preview Clustering with Small ε")
plt.xlabel("Standardized x1")
plt.ylabel("Standardized x2")
plt.show()


### Interpretation

With a smaller neighborhood radius, only the densest regions remain clustered,
while boundary and sparse points are labeled as noise. This preview reinforces
the density-based intuition of DBSCAN and illustrates the tradeoff controlled by
$\varepsilon$.

## EDA Summary

Exploratory analysis reveals a dataset composed of multiple dense, irregularly
shaped clusters with varying local density. k-nearest neighbor distance analysis
provides empirical guidance for selecting $\varepsilon$, while density
visualization explains DBSCAN’s noise assignments. These findings strongly
justify the use of DBSCAN over centroid-based clustering methods.

## Applying DBSCAN


In [ ]:
dbscan = DBSCAN(
    eps=0.3,
    min_samples=10
)

labels = dbscan.fit_predict(X_std)


## Understanding Cluster Labels

In [ ]:
unique_labels, counts = np.unique(labels, return_counts=True)
dict(zip(unique_labels, counts))


## Visualizing DBSCAN Results

In [ ]:
plt.figure(figsize=(7, 6))

scatter = plt.scatter(
    X_std[:, 0],
    X_std[:, 1],
    c=labels,
    cmap="tab10",
    s=25
)

plt.title("DBSCAN Clustering — Aggregation Dataset")
plt.xlabel("Standardized x1")
plt.ylabel("Standardized x2")
plt.colorbar(scatter, label="Cluster Label")
plt.show()


## Interpretation of Results

DBSCAN successfully identifies:
- Multiple arbitrarily shaped clusters
- Clear separation of dense regions
- Boundary and sparse points as noise (-1)

Cluster boundaries follow data geometry, not centroid distance.

This behavior highlights DBSCAN’s advantage over centroid-based methods.

## Sensitivity to $\varepsilon$

DBSCAN is sensitive to the choice of $\varepsilon$:

Smaller $\varepsilon$ → more noise points

Larger $\varepsilon$ → cluster merging

In [ ]:
dbscan_small = DBSCAN(eps=0.2, min_samples=10)
labels_small = dbscan_small.fit_predict(X_std)

plt.figure(figsize=(6, 5))
plt.scatter(
    X_std[:, 0],
    X_std[:, 1],
    c=labels_small,
    cmap="tab10",
    s=20
)
plt.title("DBSCAN with Smaller ε")
plt.show()


## 13. Comparison with K-Means (Conceptual)

The table below summarizes the conceptual differences between K-Means and DBSCAN.

| Method  | Needs $k$ | Handles Noise | Cluster Shape |
|--------|-----------|---------------|---------------|
| K-Means | Yes | No | Spherical |
| DBSCAN | No | Yes | Arbitrary |

The Aggregation dataset strongly favors DBSCAN due to its density-based
structure and irregular cluster geometry, which violates the assumptions
of centroid-based methods such as K-Means.


## Limitations

DBSCAN may struggle when:
- Cluster densities vary significantly
- Data is high-dimensional
- $\varepsilon$ is difficult to tune

Possible alternatives:
- HDBSCAN
- Spectral clustering
- Graph-based community detection

## Conclusion

In this notebook, we applied DBSCAN to the UCI Aggregation dataset using a
from-scratch implementation from the rice_ml package.

Key takeaways:
- DBSCAN identifies clusters based on density
- Noise points are naturally detected
- No prior knowledge of cluster count is required
- Arbitrary cluster shapes are handled effectively

This example reinforces DBSCAN’s strengths in density-based
unsupervised learning.